In [ ]:
import os
os.chdir("..")
print("Current Directory:", os.getcwd())

In [ ]:
def get_fluo_paths(root_path:str, mode:str="Brightfield"):
    """
    Extract paths to .nd2 files and corresponding TIFF files from the specified mode folder.

    Args:
        root_path (str): The root directory to search.
        mode (str): The folder name to focus on (default is 'Brightfield').

    Returns:
        tuple: Two lists - list of .nd2 file paths and list of tuples with corresponding TIFF file paths.
    """

    target_files = []

    for dirpath, dirnames, filenames in os.walk(root_path):
        if os.path.basename(dirpath) == mode:

            # Generate TIFF file paths dynamically based on the prefix
            cy5_path = os.path.join(dirpath, f'Captured Cy5.tif')
            fitc_path = os.path.join(dirpath, f'Captured FITC.tif')
            tritc_path = os.path.join(dirpath, f'Captured TRITC.tif')
            target_files.append((cy5_path, fitc_path, tritc_path))
            
            # Ensure all three TIFF files exist
            assert all(os.path.exists(path) for path in [cy5_path, fitc_path, tritc_path])
        
            
    return target_files

In [ ]:
base_path = os.path.join("dataset", "2024_11_29","chip_02")
out = get_fluo_paths(base_path, 'Brightfield')

In [ ]:
from src.data_processing.utils import Utils
Utils.generate_np_masks(out,seg_args=None,seg_method="comdet")